<a href="https://colab.research.google.com/github/ananyaikkurti-hub/Macroeconomic-Forecasting-Dashboard/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandas numpy scikit-learn matplotlib seaborn

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [5]:
from google.colab import files
uploaded = files.upload()

Saving gdp.csv to gdp.csv
Saving inflation.csv to inflation.csv
Saving interest.csv to interest.csv
Saving unemployment.csv to unemployment.csv


In [6]:
#skip the first 4 rows to reach the actual data.
gdp = pd.read_csv("gdp.csv", skiprows=4)
inflation = pd.read_csv("inflation.csv", skiprows=4)
interest = pd.read_csv("interest.csv", skiprows=4)
unemployment = pd.read_csv("unemployment.csv", skiprows=4)
gdp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,1.234335,3.493430,3.212471,1.225112,-23.897990,14.730616,10.636431,7.706798,6.810777,NaN
1,Africa Eastern and Southern,AFE,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,0.418937,7.937038,5.623764,4.649241,5.138168,...,2.130117,2.677524,2.705194,2.030077,-2.817572,4.578772,3.722717,1.931160,2.763839,NaN
2,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,2.260314,2.647003,1.189228,3.911603,-2.351101,-20.738839,-6.240172,2.266944,NaN,NaN
3,Africa Western and Central,AFW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,1.869593,3.726090,7.038388,5.364089,4.105339,...,0.194316,2.296349,2.904664,3.281683,-3.730630,2.549691,4.472795,3.662428,4.585674,NaN
4,Angola,AGO,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.700530,-0.149396,-0.594411,-0.204680,-4.042447,2.102753,4.216003,1.263308,4.423907,NaN


In [7]:
# to check how many countries are pres
print("Number of rows:", gdp.shape[0])
print("Number of unique countries:", gdp["Country Name"].nunique())


Number of rows: 266
Number of unique countries: 266


In [8]:
 # creating cleaning function
def clean_worldbank(df, value_name):
    # drop useless columns
    df = df.drop(columns=["Indicator Name", "Indicator Code"])
    # convert wide years into long format
    df = df.melt(
        id_vars=["Country Name", "Country Code"],
        var_name="Year",
        value_name=value_name
    )

    return df

In [9]:
gdp = pd.read_csv("gdp.csv", skiprows=4)

In [10]:
gdp_clean = gdp.drop(columns=["Indicator Name", "Indicator Code"])

gdp_clean = gdp_clean.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="Year",
    value_name="GDP"
)

gdp_clean.head()


,Country Name,Country Code,Year,GDP
0,Aruba,ABW,1960,NaN
1,Africa Eastern and Southern,AFE,1960,NaN
2,Afghanistan,AFG,1960,NaN
3,Africa Western and Central,AFW,1960,NaN
4,Angola,AGO,1960,NaN


In [11]:
inflation = pd.read_csv("inflation.csv", skiprows=4)
inflation.head()


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,6.446877,6.221375,4.689806,4.102851,5.191629,6.824727,10.883478,7.399186,4.489789,NaN
2,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,4.383892,4.975952,0.626149,2.302373,5.601888,5.133203,13.712102,-4.644709,-6.601186,NaN
3,Africa Western and Central,AFW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,1.487416,1.725486,1.784050,1.983092,2.490378,3.745568,7.949251,5.221168,3.608044,NaN
4,Angola,AGO,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,30.694415,29.844480,19.628938,17.080954,22.271539,25.754295,21.355290,13.644102,28.240495,NaN


In [12]:
inflation_clean = inflation.drop(columns=["Indicator Name", "Indicator Code"])

inflation_clean = inflation_clean.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="Year",
    value_name="Inflation"
)


In [13]:
inflation_clean.head()

,Country Name,Country Code,Year,Inflation
0,Aruba,ABW,1960,NaN
1,Africa Eastern and Southern,AFE,1960,NaN
2,Afghanistan,AFG,1960,NaN
3,Africa Western and Central,AFW,1960,NaN
4,Angola,AGO,1960,NaN


In [14]:
inflation_clean["Year"] = pd.to_numeric(inflation_clean["Year"], errors="coerce")


In [15]:
inflation_clean = inflation_clean.dropna()


In [16]:
unemployment_clean = unemployment.drop(columns=["Indicator Name", "Indicator Code"])

unemployment_clean = unemployment_clean.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="Year",
    value_name="Unemployment"
)

unemployment_clean["Year"] = pd.to_numeric(unemployment_clean["Year"], errors="coerce")
unemployment_clean = unemployment_clean.dropna()

In [17]:
interest_clean = interest.drop(columns=["Indicator Name", "Indicator Code"])

interest_clean = interest_clean.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="Year",
    value_name="Interest_value"
)

interest_clean["Year"] = pd.to_numeric(interest_clean["Year"], errors="coerce")
interest_clean = interest_clean.dropna()

In [18]:
gdp_clean["Year"] = pd.to_numeric(gdp_clean["Year"], errors="coerce")


In [19]:
df = gdp_clean.merge(
    inflation_clean,
    on=["Country Name", "Country Code", "Year"]
)

df = df.merge(
    unemployment_clean,
    on=["Country Name", "Country Code", "Year"]
)

df = df.merge(
    interest_clean,
    on=["Country Name", "Country Code", "Year"]
)


In [20]:
#final clean
df = df.dropna()
df.head()

,Country Name,Country Code,Year,GDP,Inflation,Unemployment,Interest_value
0,Australia,AUS,1991.0,-0.384631,3.176675,9.586,10.086471
1,Burundi,BDI,1991.0,4.996836,8.996939,3.147,8.348478
2,Bangladesh,BGD,1991.0,3.485228,6.357364,2.200,11.718987
3,Bahrain,BHR,1991.0,11.229998,0.764420,1.047,11.474869
4,"Bahamas, The",BHS,1991.0,-4.181785,7.114681,12.170,6.282837


In [21]:
#rechecking rows
print("Countries:", df["Country Name"].nunique())
print("Rows:", df.shape[0])

Countries: 137
Rows: 3457


In [22]:
#PHASE 4 — Feature Selection
X = df[["Inflation", "Unemployment", "Interest_value"]]
y = df["GDP"]

In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [24]:
#Train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [25]:
#Train linear regression
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [26]:
#Evaluating R^2
from sklearn.metrics import r2_score

y_pred = model.predict(X_test)

print("R2 Score:", r2_score(y_test, y_pred))

R2 Score: 0.008400550230943993


In [27]:
#Checking coefficients to check whether the inflation negativley affects GDP and whether reduces growth
coefficients = pd.DataFrame(
    model.coef_,
    X.columns,
    columns=["Coefficient"]
)
coefficients

,Coefficient
Inflation,-0.106741
Unemployment,-0.577405
Interest_value,0.219947


In [28]:
#Saving model
import pickle
pickle.dump(model, open("model.pkl", "wb"))
pickle.dump(scaler, open("scaler.pkl", "wb"))
df.to_csv("clean_macro_data.csv", index=False)

In [29]:
from google.colab import files

files.download("model.pkl")
files.download("scaler.pkl")
files.download("clean_macro_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
print(df.columns)

Index(['Country Name', 'Country Code', 'Year', 'GDP', 'Inflation',
       'Unemployment', 'Interest_value'],
      dtype='object')


In [34]:
import numpy as np

# GDP growth
df["GDP_growth"] = df.groupby("Country Name")["GDP"].pct_change()

# Lag
df["GDP_growth_lag1"] = df.groupby("Country Name")["GDP_growth"].shift(1)

# Clean
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df["GDP_growth"] = df["GDP_growth"].clip(-0.3, 0.3)

df = df.dropna()

In [35]:
print("GDP_growth_lag1" in df.columns)

True


In [37]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import numpy as np
import pandas as pd

# Ensure df exists before proceeding
features = ["Inflation", "Interest_value", "Unemployment", "GDP_growth_lag1"]

# Prepare data
X = df[features]
y = df["GDP_growth"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

tscv = TimeSeriesSplit(n_splits=5)

r2_scores = []
mae_scores = []
rmse_scores = []

for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = Ridge(alpha=1.0)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    r2_scores.append(r2_score(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))
    rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))

# Average performance
r2 = np.mean(r2_scores)
mae = np.mean(mae_scores)
rmse = np.mean(rmse_scores)

In [31]:
coefficients = pd.DataFrame({
    "Feature": ["Inflation", "Interest rate", "Unemployment"],
    "Coefficient": model.coef_
})
coefficients

,Feature,Coefficient
0,Inflation,-0.106741
1,Interest rate,-0.577405
2,Unemployment,0.219947


In [32]:
df = df.dropna()

# One-hot encode country
df_encoded = pd.get_dummies(df, columns=["Country Name"], drop_first=True)

In [39]:
X = df_encoded.drop(columns=["GDP"])
y = df_encoded["GDP"]

In [40]:
from sklearn.preprocessing import StandardScaler

num_cols = ["Inflation", "Interest_value", "Unemployment"]

scaler = StandardScaler()
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])

In [41]:
df_encoded.dtypes

,0
Country Code,object
Year,float64
GDP,float64
Inflation,float64
Unemployment,float64
...,...
Country Name_Viet Nam,bool
Country Name_West Bank and Gaza,bool
"Country Name_Yemen, Rep.",bool
Country Name_Zambia,bool


In [42]:
df_encoded = pd.get_dummies(df, columns=["Country Name"], drop_first=True)

# Convert everything to numeric safely
df_encoded = df_encoded.apply(pd.to_numeric, errors="coerce")

In [43]:
df["Year"] = pd.to_numeric(df["Year"])
df = df.sort_values(["Country Name", "Year"])

In [44]:
#Create Lag Variable
df["GDP_lag1"] = df.groupby("Country Name")["GDP"].shift(1)

In [45]:
df = df.dropna()

In [46]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# Features including lag
X = df[["Inflation", "Interest_value", "Unemployment", "GDP_lag1"]]
y = df["GDP"]

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Ridge
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)

pred = ridge.predict(X_test)
r2 = r2_score(y_test, pred)

print("Lag Model R²:", r2)

Lag Model R²: -0.06089985346846327


In [47]:
#GDP_GROWTH
# Create GDP growth
df["GDP_growth"] = df.groupby("Country Name")["GDP"].pct_change()

# Add lag of growth (very powerful)
df["GDP_growth_lag1"] = df.groupby("Country Name")["GDP_growth"].shift(1)

df = df.dropna()

In [48]:
X = df[["Inflation", "Interest_value", "Unemployment", "GDP_growth_lag1"]]
y = df["GDP_growth"]

In [49]:
pip install streamlit plotly scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.0 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.1
    Uninstalling cachetools-7.0.1:
      Successfully uninstalled cachetools-7.0.1
